# 01 - Bronze Ingestion

In [ ]:
import os

# Config
os.environ["AZURE_TENANT_ID"] = "id"
os.environ["AZURE_CLIENT_ID"] = "id"
os.environ["AZURE_CLIENT_SECRET"] = "id"

os.environ["STEAM__API_KEY"] = "id"
os.environ["FABRIC__WORKSPACE_ID"] = "id"
os.environ["FABRIC__BRONZE_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__SILVER_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__GOLD_LAKEHOUSE_ID"] = "id"

# Games to track
APP_IDS = [
    1091500,
    570,
    730,
    1245620,
    292030,
    # Add more as needed
]

print(f"Games to ingest: {len(APP_IDS)}")

In [ ]:
import sys
sys.path.insert(0, "/lakehouse/default/Files")

import asyncio
from src.steam_analytics.ingestion.orchestrator import IngestionOrchestrator, OutputTarget

orchestrator = IngestionOrchestrator(target=OutputTarget.ONELAKE)
result = await orchestrator.run(APP_IDS)

## Results

In [ ]:
from notebookutils import mssparkutils
import json

# Output visual
print("=" * 50)
print("BRONZE INGESTION COMPLETE")
print("=" * 50)
print(f"Run ID:         {result.run_id}")
print(f"Duration:       {result.duration_seconds:.2f}s")
print(f"Batches:        {len(result.batches_written)}") # Ajuste visual
# Nota: Quitamos el '%' porque tu success_rate ya viene multiplicado por 100 (100.0)
print(f"Success Rate:   {result.success_rate}%") 

if result.errors:
    print(f"Errors:         {len(result.errors)}")
    for err in result.errors[:5]:
        print(f"  - {err}")

mssparkutils.notebook.exit(json.dumps({
    "status": "success" if result.success_rate >= 80.0 else "partial",
    "run_id": str(result.run_id),
    "batches_written": len(result.batches_written),
    "success_rate": result.success_rate,
}, default=str))